Un GAN (Generative Adversarial Network) est constitué de deux réseaux neuronaux qui s’affrontent dans un processus d’apprentissage :

Le générateur : il crée des données synthétiques à partir d’un bruit aléatoire. Son objectif est de produire des données qui ressemblent le plus possible aux données réelles.
Le discriminateur : il tente de distinguer les données réelles des données générées par le générateur.
Ces deux réseaux sont en compétition :

Le générateur essaie de tromper le discriminateur en produisant des données de plus en plus réalistes.
Le discriminateur s'améliore progressivement pour mieux différencier les vraies données des fausses.
Ce processus se poursuit jusqu’à ce que le générateur parvienne à créer des données si réalistes que le discriminateur ne puisse plus les distinguer des vraies données.

CTGAN (Conditional Tabular GAN) est un modèle spécialisé dans la génération de données tabulaires synthétiques. Contrairement aux GANs classiques, il prend en compte la nature discrète et continue des variables dans un tableau, ce qui le rend plus efficace pour les bases de données structurées.

In [3]:
pip install lightgbm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 54.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install tensorflow

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
#from sklearn.metrics import precision_score, recall_score, f1_score,\
                            #accuracy_score, balanced_accuracy_score,classification_report,\
                            #plot_confusion_matrix, confusion_matrix
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.model_selection import train_test_split

import lightgbm as lgb
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, Concatenate
from tensorflow.keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
import tensorflow.keras.backend as K
from sklearn.utils import shuffle

np.random.seed(1635848)

2025-03-05 19:49:57.644711: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-05 19:49:57.649234: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-05 19:49:57.661342: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741204197.680916   11573 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741204197.686860   11573 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-05 19:49:57.708542: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [ ]:
class cGAN():
    
    """
    "Classe contenant 3 méthodes (et init) : générateur, discriminateur et entraînement. 
    Le générateur est entraîné en utilisant du bruit aléatoire 
    et un label comme entrées. Le discriminateur est entraîné en utilisant 
    des échantillons réels/faux et des labels comme entrées."
    
    """
    
    def __init__(self,latent_dim=32, out_shape=14):
        
        self.latent_dim = latent_dim
        self.out_shape = out_shape 
        self.num_classes = 2
        # Utilisation de Adam comme optimizer
        optimizer = Adam(0.0002, 0.5) #mettre à jour les poids d'un réseau de neurones pendant l'entraînement
        
        # construction du discriminateur
        self.discriminator = self.discriminator()
        #définission de la fonction de perte, l'optimiseur et les métriques à utiliser pour l'entraînement
        self.discriminator.compile(loss=['binary_crossentropy'],
                                   optimizer=optimizer,
                                   metrics=['accuracy'])

        # Construction du générateur
        self.generator = self.generator()
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
         #le générateur prend à la fois le bruit aléatoire et le label comme entrée pour générer des échantillons (gen_samples)
        gen_samples = self.generator([noise, label]) 
        
        # désactivation de l'entraînement du discriminateur lorsque le générateur est entraîné
        self.discriminator.trainable = False

        #probabilité qui indique si le discriminateur pense que l'échantillon généré est réel
        valid = self.discriminator([gen_samples, label])

        #tentative du générateur de tromper le discriminateur en produisant des échantillons aussi réalistes que possible

        # combinaison des 2 modèles
        self.combined = Model([noise, label], valid)
        
        self.combined.compile(loss=['binary_crossentropy'],
                              optimizer=optimizer,
                             metrics=['accuracy'])

    def generator(self):
        #initialise les poids des couches du générateur en utilisant une distribution normale avec une moyenne de 0 et un écart type de 0.02
        init = RandomNormal(mean=0.0, stddev=0.02)
        # les couches seront empilées les unes après les autres de manière linéaire.
        model = Sequential()

        # Une couche de 128 neurones est crée
        model.add(Dense(128, input_dim=self.latent_dim))
        # 20% des neurones sont éteints lors de l'entrainement 
        model.add(Dropout(0.2))
        model.add(LeakyReLU(alpha=0.2)) #fonction d'activation
        model.add(BatchNormalization(momentum=0.8))
        
        # Ajout de nouvelles couches
        model.add(Dense(256))
        model.add(Dropout(0.2))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Dense(512))
        model.add(Dropout(0.2))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        #couche finale
        model.add(Dense(self.out_shape, activation='tanh'))

        #création du bruit et du label conditionnel
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        #transforme les labels en vecteurs d'embedding de taille latent_dim
        label_embedding = Flatten()(Embedding(self.num_classes, self.latent_dim)(label))
        
        model_input = multiply([noise, label_embedding])
        gen_sample = model(model_input)

        return Model([noise, label], gen_sample, name="Generator")

    #De même on crée le discriminateur

    def discriminator(self):
        init = RandomNormal(mean=0.0, stddev=0.02)
        model = Sequential()

        model.add(Dense(512, input_dim=self.out_shape, kernel_initializer=init))
        model.add(LeakyReLU(alpha=0.2))
        
        model.add(Dense(256, kernel_initializer=init))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        
        model.add(Dense(128, kernel_initializer=init))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        
        #dernière couche a 1 neurone
        model.add(Dense(1, activation='sigmoid'))
        
        gen_sample = Input(shape=(self.out_shape,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, self.out_shape)(label))
        model_input = multiply([gen_sample, label_embedding])

        #donne score entre 0 et 1
        validity = model(model_input)

        return Model(inputs=[gen_sample, label], outputs=validity, name="Discriminator")
    

    def train(self, X_train, y_train, pos_index, neg_index, epochs, sampling=False, batch_size=32, sample_interval=100, plot=True): 
        
       #Les listes vont stocker les pertes du générateur et du discriminateur à chaque époque
        global G_losses
        global D_losses
        
        G_losses = []
        D_losses = []
        # les échantillons réels, utilisés pour entraîner le discriminateur
        valid = np.ones((batch_size, 1))
        #échantillons générés, utilisés pour entraîner le discriminateur considérs comme faux
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):
            
            # si sampling==True --> entrainer le  discriminator avec un batch de 8 échantillons de  postivite class et le reste negative class
            if sampling:
                idx1 = np.random.choice(pos_index, 8) #échantillon réel pour le discriminateur
                idx0 = np.random.choice(neg_index, batch_size-8) #échantillon faux ou générés
                idx = np.concatenate((idx1, idx0))

            # if sampling!=True --> les échantillons sont choisis aléatoirement parmi l'ensemble d'entraînement pour créer un batch de taille 32
            else:
                idx = np.random.choice(len(y_train), batch_size)

            #récupération des échantillons et des labels à partir des indices
            samples, labels = X_train[idx], y_train[idx]
            # mélange
            samples, labels = shuffle(samples, labels)
            
            # Génération du bruit
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            #génération de échantillons "faux" par le générateur
            gen_samples = self.generator.predict([noise, labels])

            # label smoothing: les labels des échantillons réels sont légèrement modifiés pour les rendre moins parfaits, et les labels des échantillons générés sont légèrement améliorés
            # pour éviter un apprentissage trop rapide et éviter que le modèle ne devienne trop confiant
            if epoch < epochs//1.5:
                valid_smooth = (valid+0.1)-(np.random.random(valid.shape)*0.1)
                fake_smooth = (fake-0.1)+(np.random.random(fake.shape)*0.1)
            else:
                valid_smooth = valid 
                fake_smooth = fake
                
            # Entraînement du discriminateur
            self.discriminator.trainable = True #activation de l'entraînement du disciminateur
            d_loss_real = self.discriminator.train_on_batch([samples, labels], valid_smooth) # Entraîner sur les échantillons réels
            d_loss_fake = self.discriminator.train_on_batch([gen_samples, labels], fake_smooth) # Entraîner sur les échantillons générés
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake) # Calculer la perte globale du discriminateur: moyenne des pertes réelles et générées, pondérée par 0.5

 
         #d_loss_real : La perte pour les échantillons réels est calculée en fonction des valid_smooth (les labels lissés des échantillons réels).
#d_loss_fake : La perte pour les échantillons générés est calculée en fonction des fake_smooth (les labels lissés des échantillons générés)."


            # Entrainement du générateur
            self.discriminator.trainable = False #désactive l'entraînement du discriminateur
            sampled_labels = np.random.randint(0,40, batch_size).reshape(-1, 1)
            # On génère des labels aléatoires (0 ou pour l'entraînement du générateur, afin qu'il apprenne à produire des échantillons qui trompent le discriminateur
            
            g_loss = self.combined.train_on_batch([noise, sampled_labels], valid)

          #Afficher les pertes
            if (epoch+1)%sample_interval==0:
                print('[%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f'
                  % (epoch, epochs, d_loss[0], g_loss[0]))
            G_losses.append(g_loss[0])
            D_losses.append(d_loss[0])
            if plot:
                if epoch+1==epochs:
                    plt.figure(figsize=(10,5))
                    plt.title("Generator and Discriminator Loss")
                    plt.plot(G_losses,label="G")
                    plt.plot(D_losses,label="D")
                    plt.xlabel("iterations")
                    plt.ylabel("Loss")
                    plt.legend()
                    plt.show()




